<a href="https://colab.research.google.com/github/MichaelArgs/Dog-Breed-Classification/blob/main/dog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instal Streamlit
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 83.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 5.8 MB/s eta 0:00:00


In [ ]:
# Import library yang dibutuhkan untuk deep learning, pengolahan gambar, dan Streamlit
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D
import numpy as np
import streamlit as st
from PIL import Image
import zipfile
import json
from google.colab import files

In [ ]:
# Unzip dataset yang berisi gambar jenis anjing
zip_path = '/content/70 dog breed.zip'  # Path ke file zip dataset
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/70 dog breed')  # Ekstraksi ke folder

In [ ]:
# Definisikan path untuk data pelatihan, validasi, dan pengujian
train_path = '/content/70 dog breed/train'
valid_path = '/content/70 dog breed/valid'
test_path = '/content/70 dog breed/test'

In [ ]:
# Data Augmentation untuk memperbanyak variasi data pelatihan
train_datagen = ImageDataGenerator(
    rescale=1./255,              # Normalisasi nilai pixel antara 0 dan 1
    rotation_range=40,           # Rotasi gambar hingga 40 derajat
    width_shift_range=0.2,       # Pergeseran lebar hingga 20%
    height_shift_range=0.2,      # Pergeseran tinggi hingga 20%
    shear_range=0.2,             # Distorsi gambar (shear) hingga 20%
    zoom_range=0.3,              # Zoom gambar hingga 30%
    horizontal_flip=True,        # Flip horizontal untuk variasi
    fill_mode='nearest'          # Mode pengisian pixel kosong
)

# Hanya rescale untuk data validasi
valid_datagen = ImageDataGenerator(rescale=1./255)

# Membuat generator data untuk pelatihan dan validasi
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(128, 128),      # Ubah ukuran gambar ke 128x128 pixel
    batch_size=32,               # Jumlah gambar per batch
    class_mode='categorical'     # Mode klasifikasi untuk lebih dari 2 kelas
)

valid_generator = valid_datagen.flow_from_directory(
    valid_path,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

Found 7946 images belonging to 70 classes.
Found 700 images belonging to 70 classes.


In [ ]:
# Inisialisasi model MobileNetV2 sebagai base model dengan bobot ImageNet
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Membuka lapisan terakhir untuk fine-tuning
for layer in base_model.layers[-50:]:  # Buka 50 lapisan terakhir untuk pelatihan
    layer.trainable = True

# Menambahkan lapisan tambahan di atas base model
x = base_model.output
x = GlobalAveragePooling2D()(x)    # Pooling global untuk meratakan fitur
x = Dense(512, activation='relu')(x)  # Lapisan Dense dengan 512 unit
x = Dropout(0.5)(x)                # Dropout untuk mengurangi overfitting
output = Dense(train_generator.num_classes, activation='softmax')(x)  # Lapisan output untuk klasifikasi

# Definisikan model lengkap dengan MobileNetV2 dan lapisan tambahan
model = Model(inputs=base_model.input, outputs=output)

# Kompilasi model dengan optimizer Adam dan learning rate kecil
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Setup callback untuk checkpoint, early stopping, dan pengurangan learning rate
checkpoint_callback = ModelCheckpoint(
    filepath="model_checkpoint.weights.h5",  # Menyimpan checkpoint setiap epoch
    save_weights_only=True,                  # Menyimpan hanya bobot model
    save_freq='epoch',
    verbose=1
)

# EarlyStopping untuk menghentikan pelatihan jika tidak ada peningkatan dalam 5 epoch
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Mengurangi learning rate jika tidak ada peningkatan dalam 3 epoch
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6, verbose=1)

# Kumpulan callbacks untuk pelatihan
callbacks = [checkpoint_callback, early_stopping, reduce_lr]


In [ ]:
# Define the model architecture (same as before)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Unfreeze some layers for fine-tuning
for layer in base_model.layers[-50:]:
    layer.trainable = True

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')
x = Dropout(0.5)(x)
output = Dense(train_generator.num_classes, activation='softmax')(x)

# Create the complete model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Load the saved weights
model.load_weights("model_checkpoint.weights.h5")
print("Weights loaded from the last checkpoint.")


<ipython-input-8-8ad276bba60d>:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(150, 150, 3))


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


ValueError: Only input tensors may be passed as positional arguments. The following argument value should be passed as a keyword argument: <Dense name=dense_2, built=False> (of type <class 'keras.src.layers.core.dense.Dense'>)

In [ ]:
# Melatih model menggunakan generator data pelatihan dan validasi
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=30,                      # Jumlah epoch
    callbacks=callbacks              # Memasukkan callbacks
)

Epoch 1/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.1130 - loss: 3.9646
Epoch 1: saving model to model_checkpoint.weights.h5
249/249 ━━━━━━━━━━━━━━━━━━━━ 602s 2s/step - accuracy: 0.1136 - loss: 3.9615 - val_accuracy: 0.6429 - val_loss: 1.3617 - learning_rate: 1.0000e-04
Epoch 2/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5278 - loss: 1.7542
Epoch 2: saving model to model_checkpoint.weights.h5
249/249 ━━━━━━━━━━━━━━━━━━━━ 567s 2s/step - accuracy: 0.5280 - loss: 1.7534 - val_accuracy: 0.7029 - val_loss: 1.1334 - learning_rate: 1.0000e-04
Epoch 3/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6737 - loss: 1.1569
Epoch 3: saving model to model_checkpoint.weights.h5
249/249 ━━━━━━━━━━━━━━━━━━━━ 551s 2s/step - accuracy: 0.6737 - loss: 1.1569 - val_accuracy: 0.7343 - val_loss: 1.0879 - learning_rate: 1.0000e-04
Epoch 4/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7169 - loss: 0.9914
Epoch 4: saving model to model_checkpoint.weights.h5
249/249 ━━

In [ ]:
# Simpan model akhir setelah pelatihan
model.save("dog_breed_classifier.h5")

# Unduh model yang sudah dilatih
files.download("dog_breed_classifier.h5")

In [ ]:
# Menyimpan nama kelas (jenis anjing) ke dalam file JSON
breed_names = train_generator.class_indices  # Mapping nama kelas ke indeks
with open("class_indices.json", "w") as f:
    json.dump(breed_names, f)

# Unduh file JSON yang berisi nama kelas
files.download("class_indices.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>